In [23]:
import requests

def get_image_url(article):
    iden, title = article

    endpoint = "https://en.wikipedia.org/w/api.php"
    
    params = {
        "action": "query",
        "format": "json",
        "prop": "pageimages",
        "pithumbsize": 300,    
        "titles": title
    }
    
    response = requests.get(endpoint, params=params)
    data = response.json()
    
    pages = data.get("query", {}).get("pages", {})
    
    if not pages:
        return {'id': iden, 'image_url': None}
    
    page_info = next(iter(pages.values()))
 
    thumbnail = page_info.get("thumbnail")
    if thumbnail and "source" in thumbnail:
        return {'id': iden, 'image_url': thumbnail["source"]}
    
    return {'id': iden, 'image_url': None}


In [24]:
import sqlite3
from concurrent.futures import ThreadPoolExecutor

conn = sqlite3.Connection('data/data.db')
cur = conn.cursor()

cur.execute('select id, title from articles')
articles = cur.fetchall()

conn.close()

urls = []

with ThreadPoolExecutor(max_workers=8) as pool:
    results = pool.map(get_image_url, articles)

for result in results:
    urls.append(result)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [29]:
import json

with open('data/links.jsonl', 'a', encoding='utf-8') as file:
    for url in urls:
        file.write(json.dumps(url) + '\n')